# Importing And Reading Data



In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
orders = pd.read_pickle('./drive/MyDrive/dataset/orders_no_dup.p')
orders

,InvoiceDate,InvoiceNo,StockCode,Description,UnitPrice,CustomerID,Country,Quantity
0,2010-12-01 08:26:00,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,4.25,17850.0,United Kingdom,6
1,2010-12-01 08:26:00,536365,22752,SET 7 BABUSHKA NESTING BOXES,7.65,17850.0,United Kingdom,2
2,2010-12-01 08:26:00,536365,71053,WHITE METAL LANTERN,3.39,17850.0,United Kingdom,6
3,2010-12-01 08:26:00,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,17850.0,United Kingdom,6
4,2010-12-01 08:26:00,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,17850.0,United Kingdom,6
...,...,...,...,...,...,...,...,...
531803,2011-12-09 12:50:00,581587,22730,ALARM CLOCK BAKELIKE IVORY,3.75,12680.0,France,4
531804,2011-12-09 12:50:00,581587,22899,CHILDREN'S APRON DOLLY GIRL,2.10,12680.0,France,6
531805,2011-12-09 12:50:00,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.15,12680.0,France,4
531806,2011-12-09 12:50:00,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.15,12680.0,France,4


# Working with data before the last 3 months to avoid data leakage

The approach is as follows:
* We exclude the last 3 months from the data and build the rfm model for user and every other feature only on the data 3 months before the last day 
* We create a label for each user whether or not that user made a purchase in the last 3 month 
* We represent each user by relevant features
* Predictions is done and the accuracy of the model is calculated


In [ ]:
day_model = orders["InvoiceDate"].max() - pd.Timedelta(value=91, unit='days')

orders = orders[orders['InvoiceDate']< day_model]

# RFM for classification model

In [ ]:
# Customer ID to object
# orders['CustomerID'] = orders['CustomerID'].astype(np.object)

In [ ]:
orders['TotalPrice'] = orders['UnitPrice'] * orders['Quantity']
orders

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,InvoiceDate,InvoiceNo,StockCode,Description,UnitPrice,CustomerID,Country,Quantity,TotalPrice
0,2010-12-01 08:26:00,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,4.25,17850.0,United Kingdom,6,25.50
1,2010-12-01 08:26:00,536365,22752,SET 7 BABUSHKA NESTING BOXES,7.65,17850.0,United Kingdom,2,15.30
2,2010-12-01 08:26:00,536365,71053,WHITE METAL LANTERN,3.39,17850.0,United Kingdom,6,20.34
3,2010-12-01 08:26:00,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,17850.0,United Kingdom,6,20.34
4,2010-12-01 08:26:00,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,17850.0,United Kingdom,6,20.34
...,...,...,...,...,...,...,...,...,...
328455,2011-09-09 12:42:00,566179,23236,STORAGE TIN VINTAGE DOILY,2.89,0.0,Israel,2,5.78
328456,2011-09-09 12:42:00,566179,23240,SET OF 4 KNICK KNACK TINS DOILY,4.15,0.0,Israel,5,20.75
328457,2011-09-09 12:42:00,566179,23433,HANGING QUILTED PATCHWORK APPLES,0.83,0.0,Israel,36,29.88
328458,2011-09-09 12:42:00,566179,47574A,ENGLISH ROSE SCENTED HANGING FLOWER,0.75,0.0,Israel,6,4.50


In [ ]:
# working only with purchases and not returns
orders = orders[orders['TotalPrice'] > 0]
orders

,InvoiceDate,InvoiceNo,StockCode,Description,UnitPrice,CustomerID,Country,Quantity,TotalPrice
0,2010-12-01 08:26:00,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,4.25,17850.0,United Kingdom,6,25.50
1,2010-12-01 08:26:00,536365,22752,SET 7 BABUSHKA NESTING BOXES,7.65,17850.0,United Kingdom,2,15.30
2,2010-12-01 08:26:00,536365,71053,WHITE METAL LANTERN,3.39,17850.0,United Kingdom,6,20.34
3,2010-12-01 08:26:00,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,17850.0,United Kingdom,6,20.34
4,2010-12-01 08:26:00,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,17850.0,United Kingdom,6,20.34
...,...,...,...,...,...,...,...,...,...
328455,2011-09-09 12:42:00,566179,23236,STORAGE TIN VINTAGE DOILY,2.89,0.0,Israel,2,5.78
328456,2011-09-09 12:42:00,566179,23240,SET OF 4 KNICK KNACK TINS DOILY,4.15,0.0,Israel,5,20.75
328457,2011-09-09 12:42:00,566179,23433,HANGING QUILTED PATCHWORK APPLES,0.83,0.0,Israel,36,29.88
328458,2011-09-09 12:42:00,566179,47574A,ENGLISH ROSE SCENTED HANGING FLOWER,0.75,0.0,Israel,6,4.50


In [ ]:
# the last date of purchase
orders["InvoiceDate"].max()

Timestamp('2011-09-09 12:42:00')

In [ ]:
import datetime as dt
today_date = dt.datetime(2011, 12, 11)

In [ ]:
rfm = orders.groupby('CustomerID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'InvoiceNo': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm

,InvoiceDate,InvoiceNo,TotalPrice
CustomerID,,,
0.0,92,1109,1101757.36
12346.0,326,1,77183.60
12347.0,130,5,2790.86
12348.0,249,3,1487.24
12350.0,311,1,334.40
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,127,1,100.21


In [ ]:
rfm.columns = ['recency', 'frequency', 'monetary']
rfm

,recency,frequency,monetary
CustomerID,,,
0.0,92,1109,1101757.36
12346.0,326,1,77183.60
12347.0,130,5,2790.86
12348.0,249,3,1487.24
12350.0,311,1,334.40
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,127,1,100.21


In [ ]:
rfm = rfm.reset_index()
rfm

,CustomerID,recency,frequency,monetary
0,0.0,92,1109,1101757.36
1,12346.0,326,1,77183.60
2,12347.0,130,5,2790.86
3,12348.0,249,3,1487.24
4,12350.0,311,1,334.40
...,...,...,...,...
3361,18280.0,278,1,180.60
3362,18281.0,181,1,80.82
3363,18282.0,127,1,100.21
3364,18283.0,96,10,1120.67


# Merging both rfm and orders

In [ ]:
## Vectorizing Description
# from sklearn.feature_extraction.text import TfidfVectorizer
# v = TfidfVectorizer(max_features=500)
# v.fit_transform(orders['Description']).toarray()

In [ ]:
# first we drop UnitPrice ,country, and stockcode
orders.drop(['UnitPrice','Country','StockCode','Description','InvoiceDate'],axis=1,inplace=True)
orders

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,InvoiceNo,CustomerID,Quantity,TotalPrice
0,536365,17850.0,6,25.50
1,536365,17850.0,2,15.30
2,536365,17850.0,6,20.34
3,536365,17850.0,6,20.34
4,536365,17850.0,6,20.34
...,...,...,...,...
328455,566179,0.0,2,5.78
328456,566179,0.0,5,20.75
328457,566179,0.0,36,29.88
328458,566179,0.0,6,4.50


In [ ]:
orders_cust = orders.groupby('InvoiceNo').agg({'CustomerID':'min','Quantity':'sum','TotalPrice':'sum'}).reset_index().drop('InvoiceNo',axis=1)
orders_cust

,CustomerID,Quantity,TotalPrice
0,17850.0,40,139.12
1,17850.0,12,22.20
2,13047.0,83,278.73
3,13047.0,15,70.05
4,13047.0,3,17.85
...,...,...,...
12982,12787.0,145,186.56
12983,18237.0,104,158.96
12984,16353.0,49,141.90
12985,0.0,233,423.73


In [ ]:
orders_cust = pd.merge(orders_cust,rfm,left_on='CustomerID',right_on='CustomerID',how='outer')
orders_cust

,CustomerID,Quantity,TotalPrice,recency,frequency,monetary
0,17850.0,40,139.12,373,34,5391.21
1,17850.0,12,22.20,373,34,5391.21
2,17850.0,12,22.20,373,34,5391.21
3,17850.0,88,259.86,373,34,5391.21
4,17850.0,88,259.86,373,34,5391.21
...,...,...,...,...,...,...
12982,13244.0,38,121.74,93,1,121.74
12983,17004.0,602,1006.40,93,1,1006.40
12984,13441.0,204,296.64,92,1,296.64
12985,13823.0,325,650.42,92,1,650.42
